**Change the mail id and password to get the alert to your mail about the cheaptest flight for the day**

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

import pandas as pd
import re
import time
import datetime

import smtplib
from email.mime.multipart import MIMEMultipart


In [2]:
browser = webdriver.Chrome(executable_path='C://chromedriver_win32/chromedriver.exe')
#to maximise the window
browser.maximize_window()

In [3]:
#to select the type of tickets
return_ticket = "//label[@id='flight-type-roundtrip-label-hp-flight']"
one_way_ticket = "//label[@id='flight-type-one-way-label-hp-flight']"
multi_ticket = "//label[@id='flight-type-multi-dest-label-hp-flight']"

### Ticket choice function

In [4]:
def ticket_chooser(ticket):
    try:
        ticket_type = browser.find_element_by_xpath(ticket)
        ticket_type.click()
    except Exception as e:
        pass

### departure country choice

In [17]:
def dep_country_chooser(dep_country):
    fly_from = browser.find_element_by_xpath("//input[@id='flight-origin-hp-flight']")
    time.sleep(0.2)
    fly_from.clear()
    time.sleep(0.3)
    fly_from.send_keys('  ' + dep_country)
    time.sleep(0.4)
#   to select the option from drop down when the origin/departure place is entered
    first_item = browser.find_element_by_xpath("//a[@id='aria-option-0']")
    time.sleep(1)
#   click the first option in drop down
    first_item.click()

## similarly for arrival country

In [18]:
def arrival_country_chooser(arrival_country):
    fly_to = browser.find_element_by_xpath("//input[@id='flight-destination-hp-flight']")
    time.sleep(0.2)
    fly_to.clear()
    time.sleep(0.3)
    fly_to.send_keys('  ' + arrival_country)
    time.sleep(0.4)
    first_item = browser.find_element_by_xpath("//a[@id='aria-option-0']")
    time.sleep(1)
    first_item.click()

**Choosing the Departure and Return Dates**

In [7]:
def dep_date_chooser(month, day, year):
#   here you need to select the appropriate input id for dep_date as it chages for different ticet types
    dep_date_button = browser.find_element_by_xpath("//input[@id='flight-departing-single-hp-flight']")
#   clear if any text present before and send the user entered values
    dep_date_button.clear()
    dep_date_button.send_keys(month + '/' + day + '/' + year)

**Here we have used Keys.BACKSPACE which simply tells Python to click backspace (to delete whatever is written in the date field). I put it in a for loop to click backspace 11 times to delete all the characters for the date in the field.**

In [8]:
def return_date_chooser(month, day, year):
    return_date_button = browser.find_element_by_xpath("//input[@id='flight-returning-hp-flight']")
    for i in range(11):
        return_date_button.send_keys(Keys.BACKSPACE)
    return_date_button.send_keys(month + '/' + day + '/' + year)

In [9]:
def search():
    search = browser.find_element_by_xpath("//button[@class='btn-primary btn-action gcw-submit']")
#     browser.implicitly_wait(10)
    close = browser.find_element_by_xpath("//button[@class='datepicker-close-btn close btn-text']")
    close.click()
#     ActionChains(browser).move_to_element(search).click(search)
    time.sleep(2)
    search.click()
    time.sleep(5)
    print('Results ready!')

### main scraping function 

In [10]:
df = pd.DataFrame()
def compile_data():
    global df
    global dep_times_list
    global arr_times_list
    global airlines_list
    global price_list
    global durations_list
    global stops_list
    global layovers_list
    
    #departure times
    dep_times = browser.find_elements_by_xpath("//span[@data-test-id='departure-time']")
    dep_times_list = [value.text for value in dep_times]
    
    #arrival times
    arr_times = browser.find_elements_by_xpath("//span[@data-test-id='arrival-time']")
    arr_times_list = [value.text for value in arr_times]
    
    #airline name
    airlines = browser.find_elements_by_xpath("//span[@data-test-id='airline-name']")
    airlines_list = [value.text for value in airlines]
    
    #prices
    prices = browser.find_elements_by_xpath("//span[@data-test-id='listing-price-dollars']")
    price_list = [re.sub('[^0-9]',"",value.text) for value in prices]
    
    #durations
    durations = browser.find_elements_by_xpath("//span[@data-test-id='duration']")
    durations_list = [value.text for value in durations]
                                   
    #stops
    stops = browser.find_elements_by_xpath("//span[@class='number-stops']")
    stops_list = [value.text for value in stops]
                                   
    #layovers
    layovers = browser.find_elements_by_xpath("//span[@data-test-id='flight-info']")
    layovers_list = [value.text for value in layovers]
    
    now = datetime.datetime.now()
    current_date = (str(now.year) + '-' + str(now.month) + '-' + str(now.day))
    current_time = (str(now.hour) + ':' + str(now.minute))
    current_price = 'price' + '(' + current_date + '---' + current_time + ')'
    
    for i in range(len(dep_times_list)):
        try:
            df.loc[i, 'departure_time'] = dep_times_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'arrival_time'] = arr_times_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'airline'] = airlines_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'duration'] = durations_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'stops'] = stops_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'layovers'] = layovers_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, str(current_price)] = price_list[i]
        except Exception as e:
            pass
    print('Excel Sheet Created!')

### enter your mail and password for getting a mail alert

In [11]:
#email credentials
username = 'xxxx@gmail.com'
password = 'xxxx'

In [12]:

def connect_mail(username, password):
    global server
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.ehlo()
    server.starttls()
    server.login(username, password)

In [13]:
#Create message template for email
def create_msg():
    global msg
    msg = '\nCurrent Cheapest flight:\n\nDeparture time: {}\nArrival time: {}\nAirline: {}\nFlight duration: {}\nNo. of stops: {}\nPrice: {}\n'.format(cheapest_dep_time,
                       cheapest_arrival_time,
                       cheapest_airline,
                       cheapest_duration,
                       cheapest_stops,
                       cheapest_price)

In [14]:
def send_email(msg):
    global message
    message = MIMEMultipart()
    message['Subject'] = 'Current Best flight'
    message['From'] = 'ur@gmail.com'
    message['to'] = 'ur@gmail.com'
    server.sendmail('ur@gmail.com','ur@gmail.com', msg)
    login=browser.find_element_by_xpath("//form[@id='continue']")

#### here I get SMTP error as I have not mentioned proper mail ID and password, so I've added it to the try exception block

In [20]:
try:
    for i in range(1):    
        link = 'https://www.expedia.com/'
        browser.get(link)
        time.sleep(2)
        #choose flights only
        flights_only = browser.find_element_by_xpath("//button[@id='tab-flight-tab-hp']")
        flights_only.click()
        ticket_chooser(one_way_ticket)
        dep_country=(input() or "bangalore")
        arv_country=(input() or "boston")
        dep_date=input("enter month/day/year ")
        if len(dep_date)==0:
            dep_date=(datetime.date.today()).strftime("%m/%d/%Y").split("/")
        else:
            dep_date=dep_date.split("/")
        # arv_date=input("enter month-day-year ").split("-")
        dep_country_chooser(dep_country)
        arrival_country_chooser(arv_country)
        dep_date_chooser(dep_date[0],dep_date[1],dep_date[2])
        # return_date_chooser(arv_date[0],arv_date[1],arv_date[2])
        # dep_date_chooser('08', '12', '2019')
        search()
        compile_data()
        #save values for email
        current_values = df.iloc[0]
        cheapest_dep_time = current_values[0]
        cheapest_arrival_time = current_values[1]
        cheapest_airline = current_values[2]
        cheapest_duration = current_values[3]
        cheapest_stops = current_values[4]
        cheapest_price = current_values[-1]
        print('run {} completed!'.format(i))
        create_msg()
        connect_mail(username,password)
        send_email(msg)
        print('Email sent!')
        df.to_excel('flights_from_{}_to_{}.xlsx'.format())
        time.sleep(3)
except smtplib.SMTPAuthenticationError:
    print('Your account name or password is incorrect, please try again using the correct stuff')



enter month/day/year 08/10/2019
Results ready!
Excel Sheet Created!
run 0 completed!
Your account name or password is incorrect, please try again using the correct stuff


Finally we get a excel sheet having data for different intervals. In my program I've mentioned for time.sleep(3) which is 3 seconds to check but you can make it for 1 hr interval for 1 month and perform price analysis for the flight tickets. 
This code also sends you mail about the cheapest flight fro that hour.